In [1]:
import torch, torchvision
import torchvision.transforms.functional as fn
torch.manual_seed(0)

import os
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [2]:
#Update os
path = "/home/kishen/documents/python_projects/dogs_v_cats/"
os.chdir(path)
from src.dataset import ImageDataset

Preprocessing

In [3]:
#Create labels: cats=0, dogs=1
labels = np.zeros(25_000)
labels[12_500:] =  1

In [4]:
#create augmentations for ImageDataset
path = "/home/kishen/documents/python_projects/dogs_v_cats/"
os.chdir(path)
from src.dataset import ImageDataset

transformations = torch.nn.Sequential(
        torchvision.transforms.Resize((300,300)),
        torchvision.transforms.RandomPerspective(p=.25),
        torchvision.transforms.RandomHorizontalFlip(p=.25)

)

#Build training dataset
training_data = ImageDataset(image_paths= "/home/kishen/documents/python_projects/dogs_v_cats/data/train/",
                            targets=labels,
                            augmentations= transformations)



In [5]:
from torch.utils.data import DataLoader

loader_params = {"batch_size": 16,
                 "shuffle": True,
                 "num_workers": 16}
train_loader = DataLoader(training_data, **loader_params)

**Training**

In [6]:
print(torch.cuda.is_available())


True


In [7]:
#set hyperparameters
epochs = 20
criterion = torch.nn.BCEWithLogitsLoss()
model_params={'lr':.0001,
             'momentum':.9}

#init model and optimizer
from src.models import ConvNet

model = ConvNet()
optimizer = torch.optim.SGD(model.parameters(), **model_params)

#move model to gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(device)

cuda


In [8]:
from sklearn.metrics import accuracy_score
def accuracy(preds, labels):
    preds = preds > 0.5
    return (labels == preds).sum().item() / labels.size(0)

epoch_loss = [0] * epochs
epoch_acc = [0] * epochs
for epoch in range(epochs):
    running_loss = 0.00
    correct = 0.00
    for i, data in enumerate(train_loader):
        
        #move data to gpu
        inputs, labels = data['image'], data['target']
        inputs, labels = inputs.to(device), labels.float().to(device)

        #zero gradients
        optimizer.zero_grad() 

        #model computations
        preds = model(inputs.type(torch.cuda.FloatTensor))
        loss = criterion(preds, labels.unsqueeze(1))
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        correct += accuracy(preds, labels)
    
    epoch_loss[epoch] = (running_loss / i) #i is the num_batches in epoch
    epoch_acc[epoch] = (correct/len(training_data))

    print("Loss for epoch%s: %s" % (str(epoch + 1), str(round(epoch_loss[epoch],3))))
    print("Accuracy for epoch%s: %s%" % (str(epoch + 1), str(epoch_acc[epoch] * 100)))
print("Training complete!")



Loss for epoch1: 0.623
Accuracy for epoch1: 0.507585
Loss for epoch2: 0.555
Accuracy for epoch2: 0.512375
Loss for epoch3: 0.517
Accuracy for epoch3: 0.51471
Loss for epoch4: 0.477
Accuracy for epoch4: 0.51509
Loss for epoch5: 0.438
Accuracy for epoch5: 0.51749
Loss for epoch6: 0.404
Accuracy for epoch6: 0.519435


KeyboardInterrupt: 

In [ ]:
#f, ax = plt.subplots(figsize = (10,10))
print(epoch_acc)
print(epoch_loss)

[tensor(0., device='cuda:0')]
[0.3304324869004468]


In [ ]:
y = torch.tensor([0,0,0,1,1,1])
y_hat = torch.tensor([1,1,1,1,1,1])

correct = torch.sum(y== y_hat) #/ len(y)
correct


tensor(3)